Django进阶实战07
====
**使用模板系统**

**这一部分对理解和熟练使用模板系统非常重要。**

在Python中使用模板系统分为三个步骤：

- 配置模板引擎。
- 将模板代码编译为模板。
- 使用上下文呈现模板。


在这些步骤中，Django项目通常依靠高级后端（封装的）API，而不是模板系统的低级API：

1. 对于TEMPLATES设置中的每个DjangoTemplates后端，Django实例化一个引擎。DjangoTemplates包装Engine并使其适应通用模板后端API。
2. django.template.loader模块提供了诸如get_template()之类的函数来加载模板。它们返回一个django.template.backends.django.Template，它包装了实际的django.template.Template。
3. 在上一步中获得的模板具有render()方法，该方法将上下文和可能的请求编组到Context中，并将呈现委托给底层模板。

# 配置模板引擎

Django在创建项目时，默认已配置好Djago集成的模板系统，如果没有特殊的需要，建议使用这个默认的模板系统。

# 加载模板--将模板代码编译为模板

创建模板的推荐方法是调用Engine的工厂方法：get_template（），select_template（）和from_string（）。

在一个Django项目中，TEMPLATES设置定义了一个DjangoTemplates引擎，可以直接实例化一个Template。

**`class Template`**

这个类存在于`django.template.Template`。

这个构造器接受一个参数 -- 原生的模板代码：

```
from django.template import Template

template = Template("My name is {{ my_name }}.")
```

**代码解析：**

这段代码很简短，但它足以说明创建模板的机制。

1. 导入django.template模块里的 `Template`类，它是一个模板构造器；

2. `Template`接受一个参数，一个模板代码字符串，这里是 "My name is {{ my_name }}." 它实际就是一段原生的模板代码；

3. Django支持用指定模板文件路径的方式来创建 Template 对象，即这个字符串也可以指向一个文本文件，文本文件的内容就是原生的模板代码；

4. `Template`将这段模板代码编译为一个 `Template` 类的实例对象。

# 模板中呈现context

一旦有了已编译的Template对象，就可以使用它呈现上下文context。可以重复使用相同的模板，使用不同的上下文对其进行多次渲染。

这就实现了网页静态部分代码的重复使用。

**`class Context(dict_=None)`**

django.template.Context的构造函数接受可选参数 - 将变量名称映射到变量值的字典。

Context()接受一个字典作为参数，生成一个Contex类对象的实例，这个Context实例可以作为render()函数的参数，用于替换模板中的变量或标签。

**Template.render(context)**

使用Context调用Template对象的render()方法来“填充”模板：


```
>>> from django.template import Context, Template
>>> template = Template("My name is {{ my_name }}.")

>>> context = Context({"my_name": "Adrian"})
>>> template.render(context)
"My name is Adrian."

>>> context = Context({"my_name": "Dolores"})
>>> template.render(context)
"My name is Dolores."
```

调用模板对象的render()方法，并传入变量context。render()方法将返回一个基于模板的呈现字符串，模板中的变量和标签会被context值替换。

Template.render(context) 返回的值是一个 Unicode 对象，不是普通的Python字符串。在框架中，Django会一直使用Unicode对象而不是普通的字符串。Django对Unicode的支持，将让你的应用程序轻松地处理各式各样的字符集，而不仅仅是基本的A-Z英文字符。

# 使用Django模板系统的步骤

- 写模板，原生的模板代码，可以是一个python字符串，也可以是一个文本文件；
- 将上一步原生模板代码作为参数传递给Template()方法，生成 Template 类的一个实例对象--编译后的模板对象；
- 创建 Context，常见的是以一个python字典作为Context()方法的参数，生成一个Context类的实例对象--编译后的上下文对象；
- 调用模板的 render() 方法，以上一步的Context实例对象为参数，渲染模板，实际上就是用Context的值替换模板中的变量或标签。

# 同一个模板使用多个Context渲染

创建了一个模板对象，就可以用它渲染多个Context，实际的表现就是，网页的外观看起来相近，只是其中的一些内容不同。

这正是Django框架代码重用、动态网页的精髓。

```
>>python manage.py shell

In [1]: from django.template import Template, Context

In [2]: t=Template('Hello, {{name}}')

In [3]: print(t.render(Context({'name':'John'})))
Hello, John

In [4]: print(t.render(Context({'name':'Julie'})))
Hello, Julie

In [5]: print(t.render(Context({'name':'Pat'})))
Hello, Pat
```